In [ ]:
import requests
import json
import pandas as pd

In [ ]:
headers =  {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
with open('results.txt') as f:
    lines = f.readlines()
start = int(lines[0])
for i in range(start, 500):
    try:
        results = requests.get(f'https://results.texas-election.com/static/data/election/47009/{i}/County.json', headers=headers).json()
    except:
        results = requests.get(f'https://results.texas-election.com/static/data/election/47009/{i-1}/County.json', headers=headers).json()
        with open('results.txt', 'w') as f:
            f.write(str(i-1))
        break


In [ ]:
ds = []
for i in results.keys():
    d = {}
    d['county'] = results[f'{i}']['N'] #County Name
    d['total_voters'] = results[f'{i}']['TV'] 
    d['per_reporting'] = results[f'{i}']['Races']['1004']['T'] / results[f'{i}']['TV']
    d['per_left'] = results[f'{i}']['TV'] - results[f'{i}']['Races']['1004']['T']
    d['orourke'] = results[f'{i}']['Races']['1004']['C']['331257']['V']
    d['per_orourke'] = results[f'{i}']['Races']['1004']['C']['331257']['PE']
    d['abbott'] = results[f'{i}']['Races']['1004']['C']['331306']['V'] 
    d['per_abbott'] = results[f'{i}']['Races']['1004']['C']['331306']['PE']
    ds.append(d)
ds

In [ ]:
df = pd.DataFrame(ds)

winning = []
margin = []
called = []

for i in range(0, len(df)):
    if df['orourke'][i] > df['abbott'][i]:
        winning.append('orourke')
        margin.append(df['orourke'][i] - df['abbott'][i])
    elif df['orourke'][i] == df['abbott'][i]:
        winning.append('tie')
        margin.append(0)
    else:
        winning.append('abbott')
        margin.append(df['abbott'][i]-df['orourke'][i])

df['winning'] = winning
df['margin'] = margin

for i in range(0, len(df)):
    if df['margin'][i] > df['per_left'][i]:
        called.append('yes')
    else:
        called.append('no')
        
df['called'] = called

df.to_csv('tx_election.csv', index=False)